# Prepare Q2 reference answers, whose questions are specified in Q1 post messages

Load roster file (for student's email and Q2 specification)

In [1]:
import os

os.getcwd()

'D:\\Classes\\00LCA\\2021\\Autolab\\A05\\Q1\\autograder2021g\\dynamic'

In [2]:
import sys
sys.path.append('..')

In [3]:
from extDynamicGrader1 import make_txt, look_up

In [11]:
roster_file = "roster.csv"

with open(roster_file, 'r') as f:
    roster_txt = f.read()

roster_lines = roster_txt.split('\n')
print(roster_lines[0])

email,student id,ref pattern,N,msg pattern,M,V1 mag,V1 unit,freq,phase,R1,R2,C1,L1,Q2 Answer(Ref),Vb(rms),phase shift


In [13]:
hkeys = roster_lines[0].split(",")
print(hkeys)

['email', 'student id', 'ref pattern', 'N', 'msg pattern', 'M', 'V1 mag', 'V1 unit', 'freq', 'phase', 'R1', 'R2', 'C1', 'L1', 'Q2 Answer(Ref)', 'Vb(rms)', 'phase shift']


Specification is loaded.

In [39]:
roster_lines[1:4]

['tatpong@kku.ac.th,633040001-1,dummy,0,./dynamic/Q2.pat,8,10,Vpp,400,-1.570796,100,150,3,40,./dynamic/Q2Ans.pat,,',
 'jitti.s@kkumail.com,603040451-9,dummy,0,./dynamic/Q2.pat,8,12,Vpp,200,0.00000,100,150,6,30,./dynamic/Q2Ans.pat,,',
 'kroek@kkumail.com,613040779-1,dummy,0,./dynamic/Q2.pat,8,12,Vpp,250,0.52360,200,150,6,50,./dynamic/Q2Ans.pat,,']

Solve Q2 (see the problem)

In [33]:
import numpy as np

In [34]:
def P2R(mag, phase):
    return mag * np.exp(1j*phase)

def R2P(x):
    return np.abs(x), np.angle(x)

In [35]:
# Test complex
x = 1+2j
y = 3 + 5j
z = -100j
z1 = 1/1j
z2 = x + y
z3 = x/(x + y)
print(z1)
print(z2)
print(z3)
print("z1=", R2P(z1))
print("z1=", P2R(*R2P(z1)))

-1j
(4+7j)
(0.27692307692307694+0.01538461538461538j)
z1= (1.0, -1.5707963267948966)
z1= (6.12323399574e-17-1j)


In [42]:
#Q1 (Example)

f = 400
V1 = P2R(5, -np.pi/2)
w = 2*np.pi*f
Z1 = 100
Z2 = 150
C1 = 3e-6
L1 = 40e-3

In [43]:
ZC = -1j * 1/(w*C1)
ZL = 1j * w * L1

In [46]:
# Use voltage divider
## (1) find parallel impedance, Zp
Zp = 1/(1/ZC + 1/(Z2 + ZL))

## (2) find Vb using voltage divider
Vb = V1 * Zp/(Zp + Z1)

In [47]:
Zp

(112.13513820770532-108.63357943108389j)

In [48]:
Vb

(-0.95623821838083423-3.132697756273461j)

In [54]:
V1

(3.0616169978683831e-16-5j)

In [61]:
def txt_sign(num):
    if num < 0:
        return "-", -num
    
    return "+", num

In [67]:
Vb_phasor = R2P(Vb)
print("Q1.1. Vb = {} angle {}".format(*Vb_phasor))
s, n = txt_sign(Vb_phasor[1])
print("Q1.2. Vb(t) = {} cos( {} t {} {} ) V".format(Vb_phasor[0], w, s, n))

phi = Vb_phasor[1] - np.angle(V1)

print("Q1.3. phase shift = {} rad".format(phi))

Vbrms = Vb_phasor[0]/np.sqrt(2)
print("Q1.4. Vb(rms) = {} V".format(Vbrms))

iab = (V1 - Vb)/Z1
iab_phasor = R2P(iab)
s, n = txt_sign(iab_phasor[1])
print("Q1.5. iab = {} cos( {} t {} {} ) A".format(iab_phasor[0], w, s, n))

iabrms = iab_phasor[0]/np.sqrt(2)
print("Q1.6. iab(rms) = {} A".format(iabrms))

ibc = Vb/(Z2 + ZL)
ibc_phasor = R2P(ibc)
s, n = txt_sign(ibc_phasor[1])
print("Q1.7. ibc = {} cos( {} t {} {} ) A".format(ibc_phasor[0], w, s, n))

ibcrms = ibc_phasor[0]/np.sqrt(2)
print("Q1.8. ibc(rms) = {} A".format(ibcrms))


Q1.1. Vb = 3.275391085420629 angle -1.8670574589045028
Q1.2. Vb(t) = 3.275391085420629 cos( 2513.2741228718346 t - 1.8670574589045028 ) V
Q1.3. phase shift = -0.29626113210960625 rad
Q1.4. Vb(rms) = 2.3160512475388932 V
Q1.5. iab = 0.020979059082137885 cos( 2513.2741228718346 t - 1.0975187217737032 ) A
Q1.6. iab(rms) = 0.014834434939892924 A
Q1.7. ibc = 0.018138912166783646 cos( 2513.2741228718346 t - 2.4575066677492723 ) A
Q1.8. ibc(rms) = 0.012826147796479887 A


Functionize the solution

In [96]:
def A5Q2(f, V1ampl, V1phase, R1, R2, C1, L1, show=False):
    V1 = P2R(V1ampl, V1phase)
    w = 2*np.pi*f
    Z1 = R1
    Z2 = R2
    ZC = -1j * 1/(w*C1)
    ZL = 1j * w * L1
    
    ## (1) find parallel impedance, Zp
    Zp = 1/(1/ZC + 1/(Z2 + ZL))

    ## (2) find Vb using voltage divider
    Vb = V1 * Zp/(Zp + Z1)
    
    Vb_phasor = R2P(Vb)
    if show:
        print("Q1.1. Vb = {} angle {}".format(*Vb_phasor))
    s, n = txt_sign(Vb_phasor[1])
    if show:
        print("Q1.2. Vb(t) = {} cos( {} t {} {} ) V".format(Vb_phasor[0], w, s, n))

    phi = Vb_phasor[1] - np.angle(V1)

    if show:
        print("Q1.3. phase shift = {} rad".format(phi))

    Vbrms = Vb_phasor[0]/np.sqrt(2)
#     print("Q1.4. Vb(rms) = {} V".format(Vbrms))

#     iab = (V1 - Vb)/Z1
#     iab_phasor = R2P(iab)
#     s, n = txt_sign(iab_phasor[1])
#     print("Q1.5. iab = {} cos( {} t {} {} ) A".format(iab_phasor[0], w, s, n))

#     iabrms = iab_phasor[0]/np.sqrt(2)
#     print("Q1.6. iab(rms) = {} A".format(iabrms))

#     ibc = Vb/(Z2 + ZL)
#     ibc_phasor = R2P(ibc)
#     s, n = txt_sign(ibc_phasor[1])
#     print("Q1.7. ibc = {} cos( {} t {} {} ) A".format(ibc_phasor[0], w, s, n))

#     ibcrms = ibc_phasor[0]/np.sqrt(2)
#     print("Q1.8. ibc(rms) = {} A".format(ibcrms))
    
    return Vb_phasor, w, Vbrms

In [97]:
A5Q2(f=400, V1ampl=5, V1phase=-np.pi/2, R1=100, R2=150, C1=3e-6, L1=40e-3, show=True)

Q1.1. Vb = 3.275391085420629 angle -1.8670574589045028
Q1.2. Vb(t) = 3.275391085420629 cos( 2513.2741228718346 t - 1.8670574589045028 ) V
Q1.3. phase shift = -0.29626113210960625 rad


((3.2753910854206292, -1.8670574589045028),
 2513.2741228718346,
 2.3160512475388932)

In [99]:
A5Q2(f=400, V1ampl=10, V1phase=0, R1=100, R2=150, C1=3e-6, L1=40e-3, show=True)

Q1.1. Vb = 6.550782170841258 angle -0.29626113210960625
Q1.2. Vb(t) = 6.550782170841258 cos( 2513.2741228718346 t - 0.29626113210960625 ) V
Q1.3. phase shift = -0.29626113210960625 rad


((6.5507821708412584, -0.29626113210960625),
 2513.2741228718346,
 4.6321024950777865)

## Go over the look-up entries

Go through every entry and produce the answers

In [100]:
def amplitude(mag, unit):
    ampl = mag
    if unit == "Vpp":
        ampl *= 0.5
    elif unit == "Vrms":
        ampl *= np.sqrt(2)
        
    return ampl

In [101]:
print(amplitude(220, "Vp"))
print(amplitude(220, "Vpp"))
print(amplitude(220, "Vrms"))

220
110.0
311.126983722


In [115]:
# Write a csv file

fh = open("A5Q2fromQ1.csv", mode='w')
line_app = roster_lines[0] + ", {}, {}, {}, {}\n".format("Vb.ampl", "w", "Vb.phase", "Vbrms")
fh.write(line_app)

for line in roster_lines[1:]:
    if len(line) < 1:
        break
    tokens = line.split(",")
    d = {}
    for i,t in enumerate(tokens):
        d[hkeys[i]] = t
    # end for t in tokens
    
    Vb_phasor, w, Vbrms = A5Q2(f=float(d['freq']), 
            V1ampl=amplitude(float(d['V1 mag']),d['V1 unit']),
            V1phase=float(d['phase']), R1=float(d['R1']), R2=float(d['R2']), 
            C1=float(d['C1'])*1e-6, L1=float(d['L1'])*1e-3)
    line_app = line + ", {}, {}, {}, {}\n".format(Vb_phasor[0], w, Vb_phasor[1], Vbrms)
#     print(line_app)
    fh.write(line_app)
    
fh.close()

    

In [117]:
fh.close()

In [118]:
fh

<_io.TextIOWrapper name='A5Q2fromQ1.csv' mode='w' encoding='cp1252'>